In [ ]:
from scipy.sparse import csr_matrix
import numpy as np
import qutip as qt
import matplotlib.pyplot as plt

from quantumScarsCheck import *
from quantumScarsPlotting import *
from quantumScarFunctions import *

# task 3: sparse matrix set up
N = 16 # <-----------------------------------------------------------------------------------------------------------------------------
basisList = binNoConsecOnesEfficient(N)
basisMap = {bitStr: i for i, bitStr in enumerate(basisList)}
basisLen = len(basisList)
flippedList = []

row = []
column = []

# flip bit hashmap
flipMap = {'0': '1', '1': '0'}

# -------------------------------
#
# create the PXP hamiltonian
#
# -------------------------------

for i in range(basisLen):

    # add padding so that search doesnt go out of range
    paddedBitStr = '00' + basisList[i] + '00'
    copyBit = list(paddedBitStr)

    # apply the sum of r P_r-1 * sigma_x * P_r+1 operator
    for j in range(1, N+1):
        
        if paddedBitStr[j-1] == '0' and paddedBitStr[j+1] == '0':
            copyBit[j] = flipMap[paddedBitStr[j]]
            flippedList.append(''.join(copyBit)[1:-1])
            copyBit = list(paddedBitStr)
        
    # adds row and column values for the sparse matrix
    for k in range(len(flippedList)):
        
        row.append(basisMap[flippedList[k]])
        column.append(i)
        
    flippedList.clear()

# list of ones for the sparse matrix
onesList = np.ones(len(row), dtype=int)

# create the sparse matrix and turn it into a Qobj
sparseHamiltonian = csr_matrix((onesList, (row, column)), shape=[basisLen, basisLen])
matrixHamiltonian = sparseHamiltonian.toarray()
matrixHamiltonian = qt.Qobj(matrixHamiltonian)

# diagonalize the sparse matrix
eigenvalues, eigenstates = matrixHamiltonian.eigenstates()

# initial state
z2_str = z2_initial(N)
z2_index = basisMap[z2_str]
psi0 = qt.basis(basisLen, z2_index)

# time evolution of sparse matrix
tlist = np.linspace(0, 400, 500)
evolState = qt.sesolve(matrixHamiltonian, psi0, tlist)

# create H1 operator for QobjEvo!
copyBasis = basisList
diagH1 = []

# -------------------------------
#
# create the driving hamiltonian
#
# -------------------------------

# switches 0s to -1s and keeps 1s the same for the copyBasis
# appends to diagH1 the dot product between each bit string and the 0 -> -1 Z2 state
for i in range(basisLen):

    bitString = list(copyBasis[i])
    bitString = [int(i) for i in bitString]

    z2bitString = list(z2_initial(N))
    z2bitString = 2 * np.array([int(i) for i in z2bitString]) - 1

    diagH1.append(np.dot(2 * np.array(bitString) - 1, z2bitString))

# rows and columns lists for diagonal positions in H1
rowH1 = [i for i in range(basisLen)]
columnH1 = [i for i in range(basisLen)]

# creates sparse matrix with diagonals as diagH1 list
H1 = csr_matrix((diagH1, (rowH1, columnH1)), shape=[basisLen, basisLen])
H1 = qt.Qobj(H1)
